# CNN Example code
## Please install required module first
### You need to install opencv, numpy, matplotlib

In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
import tensorflow as tf
import keras
import pandas as pd
import sklearn as sk
import scipy as sp

import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.15.133.1-microsoft-standard-WSL2-build+-x86_64-with-glibc2.35
Tensor Flow Version: 2.15.0
Keras Version: 2.15.0

Python 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Pandas 2.1.4
Scikit-Learn 1.3.2
SciPy 1.11.4
GPU is NOT AVAILABLE


# Open file \& Read image

In [42]:
img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)

In [1]:
"""for _i, _content in enumerate(y_train):
    if _i % 10 == 0 and _i != 0:
        print()
    print(_content, end=", ")"""

'for _i, _content in enumerate(y_train):\n    if _i % 10 == 0 and _i != 0:\n        print()\n    print(_content, end=", ")'

In [44]:
filter = np.array([[0,1,0],[0,1,1],[1,0,0]], dtype=np.uint8)
len(filter.shape)

2

In [45]:
class Network:
    def __init__(self, trian) -> None:
        self.istrain = trian

    def conv_layer(self, filter, stride, padding):
        if len(filter.shape) != 2:
            raise Exception("Filter is not 2 dimension array")
        if self.istrain:
            print("Running Trian process")
        else:
            print("Running Test process")
    
    def pooling_layer(self):
        pass
    
    def fcn(self):
        pass

    def loss(self):
        
        pass

    def activation(self):
        # ReLU activation function
        pass

In [46]:
cnn = Network(trian=True)
try:
    cnn.conv_layer(filter=filter, stride=_, padding=_)
except Exception as e:
    print(e)

Running Trian process
